# Improving Company's Collaboration
An analysis of a company's six months of information exchange between employees and departments. The communications were between six departments (Sales, Operations, IT, Admin, Marketing and Operations) and a total of _664_ employee's in all the departments. The analysis involved creating a network visualization of the messages sent by each employee from a department to another employee in the same or different department. This was to observe the collaborations between employees/departments and also find out areas to improve collaboration within the department or with another department.

Extensive analysis was performed to determine the most active department when sending and receiving messages, in which the **_Sales_** department was the most active in both scenarios with message count of **_1549_** and **_1227_** when sending and receiving messages for the six months period, respectively, while the **_Marketing_** department was the least active on both scenarios with message count of **_16_** and **_140_** when sending and receiving messages, respectively, for the period under study.

The results also showed that the employee with id **_598_**, aside being among the top five most influential employee (including id's _128, 605 and 586_) also has the most connections. Whilst the **Sales** department is also the most influential department, more collaborative measures should be implemented by the HR to improve collaboration in the **IT**, **Marketing** and also the **Engineering** departments.

The visualization of the messages sent and received per department within the six months period under study showed a huge decline as the month progressed as shown in the trend plot. More messages were shared between departments in the 6th month than in other months, while the 11th month had the least messages.

Below is a description of the dataset used for this study.
#### Messages has information on the sender, receiver, and time.
- "sender" - represents the employee id of the employee sending the message.
- "receiver" - represents the employee id of the employee receiving the message.
- "timestamp" - the date of the message.
- "message_length" - the length in words of the message.

#### Employees has information on each employee;
- "id" - represents the employee id of the employee.
- "department" - is the department within the company. 
- "location" - is the country where the employee lives.
- "age" - is the age of the employee.

In [1]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import networkx as nx
import warnings

%matplotlib inline
sb.set_theme(style = 'whitegrid')
warnings.filterwarnings('ignore')

In [2]:
# Loading datasets
messages = pd.read_csv('data/messages.csv', parse_dates = ['timestamp'])
employees = pd.read_csv('data/employees.csv')

In [3]:
# Copying data
messages_sent = messages.copy()
employees_data = employees.copy()

In [4]:
# Checking for missing values
print('Message data info:')
print(messages_sent.isnull().sum())
print('\nEmployee data info:')
print(employees_data.isnull().sum())

Message data info:
sender            0
receiver          0
timestamp         0
message_length    0
dtype: int64

Employee data info:
id            0
department    0
location      0
age           0
dtype: int64


The `messages_sent` and `employees_data` have no missing values as observed from the results obtained above. The columns also have appropriate data types assigned to them.

One issue arises, though trivial but important, sender and receiver in the `messages_sent` data both indicate the sender and receiver of the message(s) respectively, but are actually the id of both, thus the columns will be renamed to `sender_id` and `receiver_id` respectively. This is to properly communicate what the feature (column) actually contains.

In [5]:
# Renaming columns
messages_sent.rename(columns = {'sender':'sender_id', 'receiver':'receiver_id'}, inplace = True)
messages_sent.head(2)

,sender_id,receiver_id,timestamp,message_length
0,79,48,2021-06-02 05:41:34,88
1,79,63,2021-06-02 05:42:15,72


#### Checking for duplicates

In [6]:
messages_sent.duplicated().sum()

3

In [7]:
employees_data.duplicated().sum()

0

**Exploring further on duplicated values**

In [8]:
# Subset duplicated values
duplicate_values = messages_sent.duplicated(keep = False)
messages_sent[duplicate_values]

,sender_id,receiver_id,timestamp,message_length
3333,1807,32,2021-10-13 22:25:17,50
3365,1657,1675,2021-11-02 07:42:25,52
3377,1881,1676,2021-11-17 06:45:28,27
3446,1807,32,2021-10-13 22:25:17,50
3478,1657,1675,2021-11-02 07:42:25,52
3490,1881,1676,2021-11-17 06:45:28,27


From all indications, the values are duplicated and thus will be dropped. 

This decision was taken because a particular sender (sender_id) cannot send the same exact message with the same message length, at the same date and same exact timestamp, to the same receiver (receiver_id). In real time, there must have been a difference in the time in seconds.

In [9]:
# Drop duplicates
messages_sent = messages_sent.drop_duplicates()

In [10]:
# check
messages_sent.duplicated().sum()

0

### Merging Datasets
The `messages_sent` and `employee_data` will be merged using the `sender_id`, `receiver_id` and `id`. In order to achieve a successful merge, the dataframe's must have a common key, in this case, sender_id, receiver_id and id.

The merge will use a `left join` as this will filter all the messages sent by each employee using the sender_id, including their details and receiver_id too.

In [11]:
# Merging data frames using 'left join' on 'sender_id' and 'id'
df_x = pd.merge(messages_sent, employees_data, how = 'left', left_on = 'sender_id', right_on = 'id')

# Merging data frames using 'left join' on 'receiver_id' and 'id'
df = pd.merge(df_x, employees_data, how = 'left', left_on = 'receiver_id', right_on = 'id')

In [13]:
# Check for null values
print('Null values summary:\n',df.isnull().sum())

# Check for duplicated values
print('\n\nThere are {} duplicated values\n\n'.format(df.duplicated().sum()))

Null values summary:
 sender_id         0
receiver_id       0
timestamp         0
message_length    0
id_x              0
department_x      0
location_x        0
age_x             0
id_y              0
department_y      0
location_y        0
age_y             0
dtype: int64


There are 0 duplicated values




In [14]:
# First five rows of merged data frame
df.head(5)

,sender_id,receiver_id,timestamp,message_length,id_x,department_x,location_x,age_x,id_y,department_y,location_y,age_y
0,79,48,2021-06-02 05:41:34,88,79,Sales,France,33,48,IT,France,34
1,79,63,2021-06-02 05:42:15,72,79,Sales,France,33,63,Sales,France,38
2,79,58,2021-06-02 05:44:24,86,79,Sales,France,33,58,Sales,Germany,40
3,79,70,2021-06-02 05:49:07,26,79,Sales,France,33,70,Operations,France,47
4,79,109,2021-06-02 19:51:47,73,79,Sales,France,33,109,Marketing,Brasil,32


**The following modifications will be made to make the dataframe suitable for our analysis:**
- department_x and department_y will be renamed to sender_department and receiver_department respectively
- location_x and location_y will be renamed to sender_location and receiver_location respectively
- age_x and age_y will be renamed to sender_age and receiver_age respectively
- A new column `month` will be created from the timestamp so the trend of messages sent and received by each department can be visualized

In [15]:
# Renaming dataframe columns:
df.rename(columns = {'department_x':'sender_department', 'department_y':'receiver_department', 'location_x':'sender_location', 
                     'location_y':'receiver_location', 'age_x':'sender_age', 'age_y':'receiver_age'}, inplace = True)

# Extract the month form the timestamp
df['month'] = df['timestamp'].dt.month

# First 2 rows of the new data frame
df.head(5)

,sender_id,receiver_id,timestamp,message_length,id_x,sender_department,sender_location,sender_age,id_y,receiver_department,receiver_location,receiver_age,month
0,79,48,2021-06-02 05:41:34,88,79,Sales,France,33,48,IT,France,34,6
1,79,63,2021-06-02 05:42:15,72,79,Sales,France,33,63,Sales,France,38,6
2,79,58,2021-06-02 05:44:24,86,79,Sales,France,33,58,Sales,Germany,40,6
3,79,70,2021-06-02 05:49:07,26,79,Sales,France,33,70,Operations,France,47,6
4,79,109,2021-06-02 19:51:47,73,79,Sales,France,33,109,Marketing,Brasil,32,6


The final dataframe obtained contains detailed information needed for our analysis. From the dataframe, details about the sender and receiver (id, department, location, and age) can easily be obtained.

The final dataframe will be saved as a master data. This is to have easy access to the data used for the analysis and avoid repeating the process before hand.


In [16]:
# Saving final data set
df.to_csv('data/master_data_set_emp_net.csv', index = False)

### Exploratory Data Analysis

In this section, we will make use of the already prepared `master_data_set` as it contains all information needed to derive insights and make decisions.

In [17]:
# Loading master data set
df = pd.read_csv('data/master_data_set_emp_net.csv', parse_dates = ['timestamp'])

### Descriptive Statistics

In [18]:
describe_df = df[['message_length', 'sender_age', 'receiver_age']]
describe_df.describe()

,message_length,sender_age,receiver_age
count,3509.000000,3509.000000,3509.000000
mean,48.455400,38.943574,41.065546
std,22.864271,8.340714,10.398350
min,10.000000,23.000000,22.000000
25%,29.000000,31.000000,33.000000
50%,49.000000,38.000000,41.000000
75%,68.000000,47.000000,50.000000
max,88.000000,59.000000,59.000000


In [21]:
int(100 * 1.1**7)

194

In [22]:
# Definition of pi_string
pi_string = "3.1415926"

# Convert pi_string into float: pi_float
pi_float = float(pi_string)

In [23]:
pi_float

3.1415926

In [24]:
True + False

1

In [26]:
"I said " + ("Hey " * 2) + "Hey!"

'I said Hey Hey Hey!'